# 线段树
给定一个数组，需要对指定区间进行三种任务(操作):
- 对区间里的数均加m(不返回数组)，
- 把区间里的数均变成m(不返回数组)，
- 计算区间里的数的所有和(返回结果)。

难点是：要求以上操作时间复杂度为O($logN$)。

### 分析：
- 由于时间复杂度要求, 我们不能在每个步骤时都去挨个操作区间中的树, 而是记录下来, 到最后需要返回求和时根据前面的两个步骤统一下发。
- 线段树的基本思想: 把一个数组变成一个满二叉树，然后对满二叉树进行操作。这个满二叉树的每个结点都表示数组中的一个子区间 / 单个元素, 
![线段树](附件图片\线段树.jpg)

- 考虑到满二叉树性质: 节点i的左右子树分别是2i和2i+1。以求和认为为例, 只需要知道当前节点的左右子树的求和，就可以知道当前节点的求和。因此我们将Sum_arr构造成一个满二叉树. 
    - 定义Sum_arr[i]: 表示第i个节点表示的子区间(单个元素)的和
    - Sum_arr[i] = Sum_arr[2i] + Sum_arr[2i+1]: 由于第i个节点表示的区间是父区间, 那么其和等于其子区间的和, 再根据满二叉树性质, 即可得到该递推结果.
    - 为了能使得上述递推式成立, 根节点应该是Sum_arr中下标为1的节点, 因此Sum_arr[1]表示的是整个数组[0, n-1]的元素和
    - 如果数组arr的长度N, 我们需要用一维数组Sum_arr来构建二叉树, 需要开辟多少空间长度呢？如果按照本例的自顶向下进行构建(现有区间再有子区间), 需要4N的长度(因为N个叶节点的满二叉树最坏的结果是有4N-5个节点); 如果是自底向上进行构建, 则只需要2N的空间大小 [参考网页](https://zhuanlan.zhihu.com/p/65943900)
- 查询区间[L, R]的和: 由于将区间分成了二叉树的节点, 因此我们可以只访问一些节点即可得到[L, R]和元素和。我们以上图为例, 如果需要求[4, 8]区间元素和, 根据上图节点我们只需要访问[4, 5]节点和[6, 8]节点, 因此只需要访问到这两个节点即可. 在比如我们需要求[5, 8]的元素和, 此时我们需要访问节点[5]、[6, 8]。因此线段树的查询操作是可以做到O($logn$)。
- 操作区间[L, R]更新: 由于线段树中最后的每个叶节点代表的都是最后的单点元素, 因此我们更新[L, R]区间时, 需要访问到所有的[L, R]的叶节点。并且更新完叶节点后, 还需要从下往上, 将受到影响的父亲节点进行更新。所以更新[L, R]区间中的元素, 时间复杂度为O($nlogn$).
    - 使用lazy机制实现任务的下发更新优化。我们考虑每次更新只更新到更新区间完全覆盖线段树结点区间为止。依然以上图为例，如果我们将[4, 8]区间中的数每个+1，更新任务下发时只下发到[4, 5]节点和[6, 8]节点, 只更新这两个节点的状态值(也就是该区间数值的和), 而不再继续往下发。但这样就会导致被更新结点的子孙结点的区间得不到需要更新的信息(比如这个例子中[4]、[5]、[6]、[7]、[8]这些叶节点的值并没有被更新)。但是虽然叶节点没有得到更新, 但我们在查询[4, 8]区间和时仍可以得到正确值。
    - 为了解决子节点没有得到更新消息这一问题，我们使用一个lazy_tag对每个节点进行标记, 其含义是当前节点已经存在任务并完成了状态更新, 但子节点还没有收到。当下次遇到需要用该节点下面的子节点的信息，再去更新该节点下面的子节点。使用lazy机制进行任务的下发更新, 时间复杂度为O($logn$)
    - 如果当前节点中已经存在了lazy_tag, 并且又接收到了一次需要下发到子节点的任务, 那么此时需要将上一个任务一起下发到子节点



## 封装类
为了实现标题中的三种任务, 我们构造出以下线段树: 
- build: 使用二分+递归的方式实现sum_arr求和线段树的构造
- self.sum_arr[i]: 节点i区间中的元素和, 节点从1开始, 节点1代表的区间为[0,n-1]
- self.lazy[i]: 表示节点i的增加数值, 即将节点i表示的区间中所有元素加上self.lazy[i]
- self.change[i]: 表示节点i的更新数值, 即将节点i表示的区间中所有元素更新为self.change[i]
- self.lazy[i]和self.change[i]其实表示的就是两个任务的lazy_tag, 如果其非零则表示该类任务还没下发给子节点
- 加数需求：通过lazy机制, 当任务区间[L, R]刚好就是本节点代表的区间时, 将不会继续向子节点下发任务, 更新节点状态, 以及lazy_tag. 需要注意的是, 如果来了一个自身“揽”不住的任务时(需要下发到子节点去执行), 但自己本身已经揽有任务(存在lazy_tag), 则需要先将已经堆积的任务先下发给子节点。
- 更新需求：通过lazy机制, 但更新任务来的优先级会将, 该节点已经“lazy”的所有加数需求全部抛弃。并且如果自己本身已经揽有任务(存在lazy_tag), 则需要先将已经堆积的任务先下发给子节点。
- 下发函数(pushDown): 在加数需求和更新需求中我们都看到需要将已经“揽”的任务下发到子树。本应该分别下发lazy住的更新任务和加数任务, 但实际上如果同时存在self.lazy[i]和self.change[i]非0, 那么一定是先下发了更新任务到本节点, 而后下发了加数任务到节点。(因为在执行更新需求时已经将本节点的加数需求清空了). 因此本例中将下发函数写到了一起, 并且一定要先下发更新任务, 再下发加数任务。
- add(self, L, R, value, l=0, r=None, rt=1): 对[L, R]区间中的每个元素都加value值, 其中rt表示当前区间[l, r]的节点, 外部调用时不需要传递这三个参数, 这三个参数会默认为从节点1开始向下查找, 节点1代表的区间为[0, n-1]
- updata(self, L, R, value, l=0, r=None, rt=1): 将[L, R]区间中的每个元素都变成value, 其中rt表示当前区间[l, r]的下标, 外部调用时不需要传递这三个参数, 这三个参数会默认为从节点1开始向下查找, 节点1代表的区间为[0, n-1]

In [1]:
class SegmentTree():
    def __init__(self, origin):
        self.MAXlEN = len(origin)
        self.arr = origin
        self.sum_arr = [0 for i in range(4 * self.MAXlEN)]   # 求和线段树, 下标i表示第i个节点
        self.lazy = [0 for i in range(4 * self.MAXlEN)]
        self.change = [None for i in range(4 * self.MAXlEN)]
        # 从第1个节点构建线段树, 第1个节点表示[0, n-1]区间
        self.build(0, len(origin)-1, 1)
    
    # 通过递归构建Sum数组, 对[l, r]区间中的元素构建线段树, rt表示在线段树的index。
    def build(self, l, r, rt):
        if l == r:   # 如果为叶节点则直接赋值
            self.sum_arr[rt] = self.arr[l]
            return
        # 以二分的方式构建左右子树
        mid = (l+r)//2
        self.build(l, mid, 2*rt)       # 建立左子树
        self.build(mid+1, r, 2*rt+1)   # 建立右子树
        # 根据子节点, 更新当前节点
        self.sum_arr[rt] = self.sum_arr[2*rt] + self.sum_arr[2*rt+1]


    # 下发本级lazy的任务信息, 下发顺序是先下发更新任务, 再下发求和任务：
    # l, r表示当前节点表示的区间范围, rt表示当前节点的index。
    def pushDown(self, l, r, rt):
        mid = (l+r)//2
        # 如果当前节点有更新任务，则下发
        if self.change[rt] != None:
            self.updata(l, mid, self.change[rt], l, mid, rt*2, )
            self.updata(mid+1, r, self.change[rt], mid+1, r, rt*2+1)
            self.lazy[rt] = 0
            self.change[rt] = None

        # 如果当前节点有求和任务，则下发
        if self.lazy[rt] != 0:
            self.add(l, mid, self.lazy[rt], l, mid, rt*2)
            self.add(mid+1, r, self.lazy[rt], mid+1, r, rt*2+1)
            self.lazy[rt] = 0
            # 写法2
            # lazy[rt*2] += lazy[rt]
            # lazy[rt*2+1] += lazy[rt]
            # Sum[rt*2] += lazy[rt] * (mid-l+1)
            # Sum[rt*2+1] += lazy[rt] * (r-mid)
            # lazy[rt] = 0


    # 加数任务: 对[L, R]区间中的每个元素都加value值
    # l, r表示当前节点表示的区间范围, rt表示当前节点的index。
    def add(self, L, R, value, l=0, r=None, rt=1):
        r = self.MAXlEN-1 if r is None else r  # 默认开始的节点是第1节点: [0, n-1]

        if L<=l and r<=R:  # 如果当前节点完全被[L,R]包含, 则直接更新当前节点, 并更新对应的lazy_tag信息
            self.lazy[rt] += value
            self.sum_arr[rt] += value*(r-l+1)    # 更新线段树中对应节点的sum信息
            return
        # 需要先将本节点已经存在的信息下发下去
        self.pushDown(l, r, rt)
        mid = (l+r)//2
        if L <= mid:  #左节点是否需要接受信息
            self.add(L, R, value, l, mid, 2 * rt)    #下发到左节点
        if mid+1 <= R:  #右节点是否需要接受信息
            self.add(L, R, value, mid+1, r, 2 * rt + 1)    #下发到右节点
        # 信息下发完成之后, 根据子节点, 更新当前节点
        self.sum_arr[rt] = self.sum_arr[2*rt] + self.sum_arr[2*rt+1]


    # 更新任务: 将[L, R]区间中的每个元素都变成value
    # l, r表示当前节点表示的区间范围, rt表示当前节点的index。
    def updata(self, L, R, value, l=0, r=None, rt=1):
        r = self.MAXlEN-1 if r is None else r  # 默认开始的节点是第1节点: [0, n-1]
        if L<=l and r<=R:
            self.change[rt] = value 
            self.sum_arr[rt] = value * (r-l+1)  # 之前的累加和全部无效了
            self.lazy[rt] = 0                   # 由于有更新任务因此之前积累的所有lazy信息都无效了
            return
        # 需要先将本节点已经存在的lazy下发下去
        self.pushDown(l, r, rt)
        mid = (l+r)//2
        if L <= mid:    # 左节点是否需要接受信息
            self.updata(L, R, value, l, mid, 2 * rt)    #下发到左节点
        if mid+1 <= R:  # 右节点是否需要接受信息
            self.updata(L, R, value, mid+1, r, 2 * rt + 1)    #下发到右节点
        # 信息下发完成之后, 根据子节点, 更新当前节点
        self.sum_arr[rt] = self.sum_arr[2*rt] + self.sum_arr[2*rt+1]


    # 查询函数: 查询[L, R]区间中的元素和
    def query(self, L, R, l=0, r=None, rt=1):
        r = self.MAXlEN-1 if r is None else r
        if L<=l and r<=R:
            return self.sum_arr[rt]
        self.pushDown(l, r, rt)
        mid = (l+r)//2
        ans = 0
        if L <= mid:  
            ans += self.query(L, R, l, mid, 2 * rt)    #向左节点要信息
        if mid+1 <= R:  
            ans += self.query(L, R, mid+1, r, 2 * rt + 1)    #向右节点要信息
        return ans


In [8]:
origin = [3, 4, 3, 4]
linetree = SegmentTree(origin)
print(linetree.sum_arr) 
linetree.add(1,3,5)         # 3, 9, 8, 9
print(linetree.query(0, 3)) # 29
linetree.updata(1,3,0)      # 3, 0, 0, 0
print(linetree.query(0, 3)) # 3
linetree.updata(2,3,1)      # 3, 0, 1, 1
print(linetree.query(0, 3)) # 5
linetree.add(1,3,2)         # 3, 2, 3, 3
print(linetree.query(0, 3)) # 11

[0, 14, 7, 7, 3, 4, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0]
29
3
5
11


In [11]:
origin = [3, 4, 3, 4]
linetree = SegmentTree(origin)
linetree.add(1,3,5)         # 3, 9, 8, 9
for i in range(4):
    print(linetree.query(i, i), end=' ')
linetree.updata(1,3,0)      # 3, 0, 0, 0
print()
for i in range(4):
    print(linetree.query(i, i), end=' ')
linetree.updata(2,3,1)      # 3, 0, 1, 1
print()
for i in range(4):
    print(linetree.query(i, i), end=' ')
linetree.add(1,3,2)         # 3, 2, 3, 3
print()
for i in range(4):
    print(linetree.query(i, i), end=' ')

3 9 8 9 
3 0 0 0 
3 0 1 1 
3 2 3 3 